# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

skip = True

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Archie_Short_CW_Folder_UG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Mounted at /content/drive
['.DS_Store', 'Personal_Dataset', 'Models', 'CW_Dataset', 'Code', 'test_functions.ipynb']


# Import and format training data

In [ ]:
TRAIN_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'CW_Dataset/train')
tData = pd.read_csv(os.path.join(TRAIN_FOLDER,"train_data_refined.csv"))
trainData = tData.to_numpy()

In [ ]:
#length of training data
print(trainData.shape) 

print(trainData[0])
#reformat input to data array, label array 
def formatTraining(trainingData):
  labels = []
  formattedTraining = []
  for sample in trainingData:
    labels.append(sample[len(sample)-1])
    sample = sample[:-1]; sample 
    formattedTraining.append(sample) 


  return formattedTraining, labels
  
from numpy.random import shuffle
shuffle(trainData)






(12271, 8501)
[ 0.11125135  0.16247456  0.07009258 ... 31.         15.
  7.        ]


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


x,y = formatTraining(trainData)




In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

import sklearn.model_selection as model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(x,y, train_size=0.80, test_size=0.20)



X_train = np.array(X_train)

X_test= np.array(X_test)


y_train= np.array(y_train)
y_test = np.array(y_test)


print(X_train.shape)
print(X_test.shape)

print(type(y_train))

(9816, 8500)
(2455, 8500)
<class 'numpy.ndarray'>


# Reshape data for CNN
The CNN requires the data to be shaped a certain way, this also applies to the label as well so I need to create some functions to convert my data into a way thats more friendly for the keras library. 

In [ ]:
import tensorflow as tf
X_train = tf.reshape(X_train,(9816,8500,1))
X_test = tf.reshape(X_test,(2455,8500,1))

#print(X_train)
#print(X_test)

def convert_labels(label_array):
  newTrain = []
  for label in label_array:
    label_vector = [0,0,0,0,0,0,0]
    label_vector[int(label)-1] = 1
    newTrain.append(np.array(label_vector))
  return np.array(newTrain) 
    
y_copy = y_test
y_train= convert_labels(y_train)

y_test = convert_labels(y_test)
#print(y_test.shape)
#y_train = np.argmax(y_train, axis=1)
#y_test = np.argmax(y_test, axis=1)
#print(y_train[0])



# CNN Model creation and Training
First we create the model then can train and fit it to the data. I'll be using a CNN model that works with a 1d input specifically. https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/

https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca 



In [ ]:
#imports for the CNN 
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


In [ ]:
#reference https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/ 
#https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca 

verbose, epochs, batch_size = 1, 5, 32
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=((8500,1))))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
accuracy = accuracy * 100.0
print("Accuracy: ",accuracy)

Epoch 1/5
307/307 [==============================] - 158s 511ms/step - loss: 1.3611 - accuracy: 0.6169
Epoch 2/5
307/307 [==============================] - 159s 517ms/step - loss: 0.6171 - accuracy: 0.7905
Epoch 3/5
307/307 [==============================] - 161s 524ms/step - loss: 0.2490 - accuracy: 0.9167
Epoch 4/5
307/307 [==============================] - 157s 511ms/step - loss: 0.0560 - accuracy: 0.9841
Epoch 5/5
307/307 [==============================] - 155s 504ms/step - loss: 0.0155 - accuracy: 0.9961
Accuracy:  70.30549645423889


In [ ]:
predictions = model.predict(X_test, verbose=0)
converted = False
if not converted:
  predictions=np.argmax(predictions,axis=1)
  for position, pred in enumerate(predictions):
    predictions[position] = pred +1
print(predictions)

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
#reference https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

print(classification_report(predictions,y_copy,zero_division=1))


[7 1 5 ... 5 2 7]
              precision    recall  f1-score   support

           1       0.95      0.96      0.96       271
           2       0.90      0.96      0.93        49
           3       0.84      0.95      0.89       135
           4       0.98      0.95      0.97       977
           5       0.93      0.93      0.93       396
           6       0.91      0.97      0.94       124
           7       0.94      0.94      0.94       503

    accuracy                           0.95      2455
   macro avg       0.92      0.95      0.94      2455
weighted avg       0.95      0.95      0.95      2455



Next I save the model to the folder if I think the results are satisfactory, I can edit the results_good after the execution

In [ ]:
MODEL_FOLDER = os.path.join(GOOGLE_DRIVE_PATH,'Models')

results_good = False
#SAVE
if results_good:
  model.save(os.path.join(MODEL_FOLDER,"CNN_Model"))

INFO:tensorflow:Assets written to: drive/My Drive/University/Computer Vision/CW_Folder_UG/Models/CNN_Model2/assets


In [ ]:
#LOAD
loaded_cnn_model = keras.models.load_model(os.path.join(MODEL_FOLDER,"CNN_Model"))
_, accuracy = loaded_cnn_model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
accuracy = accuracy * 100.0
print("Accuracy: ",accuracy)

77/77 [==============================] - 8s 103ms/step - loss: 0.3016 - accuracy: 0.9462
Accuracy:  94.62321996688843
